In [ ]:
## 学习要点

1、正则表达式在spaCy中的应用

2、

In [1]:
# 导入正则表达式模块
import re

In [2]:
# 定义一个包含两个人名的文本字符串
text = "Paul Newman was an American actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."

In [3]:
# 定义一个正则表达式模式,用于匹配以Paul开头的名字
pattern = r"Paul [A-Z]\w+"

In [4]:
# 使用finditer方法在文本中查找所有匹配的内容
matches = re.finditer(pattern, text)
# 遍历并打印所有匹配的结果
for match in matches:
    print(match)

<re.Match object; span=(0, 11), match='Paul Newman'>
<re.Match object; span=(39, 53), match='Paul Hollywood'>


In [5]:
# 导入spacy库和Span类
import spacy
from spacy.tokens import Span

In [6]:
# 创建一个空白的英语语言模型
nlp = spacy.blank("en")
# 使用nlp对象处理文本
doc = nlp(text)

In [7]:
# 从spacy.language模块导入Language类
from spacy.language import Language

In [9]:
# 定义一个自定义的命名实体识别（NER）组件，专门用于识别以"Paul"开头的人名
@Language.component("paul_ner")
def paul_ner(doc):
    # 定义正则表达式模式：匹配"Paul"后跟一个大写字母和任意数量的字母
    pattern = r"Paul [A-Z]\w+"
    
    # 获取文档中已存在的命名实体列表
    original_ents = list(doc.ents)
    
    # 初始化一个新的列表，用于存储找到的"Paul"实体
    mwt_ents = []
    
    # 使用正则表达式在文本中查找所有匹配的"Paul"名字
    for match in re.finditer(pattern, doc.text):
        # 获取匹配文本的起始和结束字符索引
        start, end = match.span()
        
        # 使用字符索引创建一个spaCy的Span对象
        # Span对象表示文档中的一段连续文本
        span = doc.char_span(start, end)
        
        # 确保创建的span是有效的（即完美对应词元边界）
        if span is not None:
            # 将新找到的实体信息（开始词元索引、结束词元索引、实体文本）添加到列表中
            mwt_ents.append((span.start, span.end, span.text))
    
    # 将新识别的"Paul"实体添加到原始实体列表中
    for ent in mwt_ents:
        start, end, name = ent
        # 创建一个新的Span对象，标记为"PERSON"类型
        per_ent = Span(doc, start, end, label="PERSON")
        # 将新创建的实体添加到原始实体列表中
        original_ents.append(per_ent)
    
    # 用包含新"Paul"实体的更新列表替换文档的原始实体列表
    doc.ents = original_ents
    
    # 返回更新后的文档对象
    return doc

In [24]:
# 创建一个新的空白英语语言模型
nlp2 = spacy.blank("en")
# 将自定义的命名实体识别组件添加到处理管道中
nlp2.add_pipe("paul_ner")

<function __main__.paul_ner(doc)>

In [25]:
# 使用新的nlp2对象处理文本
doc2 = nlp2(text)
# 打印识别出的实体
print(doc2.ents)

(Paul Newman, Paul Hollywood)


In [30]:
# 从spacy.util模块导入filter_spans函数
from spacy.util import filter_spans

In [31]:
# 定义另一个自定义的命名实体识别（NER）组件，专门用于识别"Hollywood"并将其标记为电影相关实体
@Language.component("cinema_ner")
def cinema_ner(doc):
    # 定义正则表达式模式：精确匹配"Hollywood"
    pattern = r"Hollywood"
    
    # 获取文档中已存在的命名实体列表
    original_ents = list(doc.ents)
    
    # 初始化一个新的列表，用于存储找到的"Hollywood"实体
    mwt_ents = []
    
    # 使用正则表达式在文本中查找所有"Hollywood"出现的位置
    for match in re.finditer(pattern, doc.text):
        # 获取匹配文本的起始和结束字符索引
        start, end = match.span()
        
        # 使用字符索引创建一个spaCy的Span对象
        span = doc.char_span(start, end)
        
        # 确保创建的span是有效的（即完美对应词元边界）
        if span is not None:
            # 将新找到的实体信息（开始词元索引、结束词元索引、实体文本）添加到列表中
            mwt_ents.append((span.start, span.end, span.text))
    
    # 将新识别的"Hollywood"实体添加到原始实体列表中
    for ent in mwt_ents:
        start, end, name = ent
        # 创建一个新的Span对象，标记为"CINEMA"类型
        per_ent = Span(doc, start, end, label="CINEMA")
        # 将新创建的实体添加到原始实体列表中
        original_ents.append(per_ent)
    
    # 使用filter_spans函数过滤重叠的span
    # 这可以解决新添加的"CINEMA"实体可能与现有实体重叠的问题
    filtered = filter_spans(original_ents)
    
    # 用过滤后的实体列表更新文档的实体
    # 这确保了文档只包含非重叠的、最优的实体集合
    doc.ents = filtered
    
    # 返回更新后的文档对象
    return doc

In [32]:
# 加载预训练的英语语言模型
nlp3 = spacy.load("en_core_web_sm")
# 将自定义的命名实体识别组件添加到处理管道中
nlp3.add_pipe("cinema_ner")

<function __main__.cinema_ner(doc)>

In [33]:
# 使用nlp3对象处理文本
doc3 = nlp3(text)
# 遍历并打印所有识别出的实体及其标签
for ent in doc3.ents:
    print(ent.text, ent.label_)

Paul Newman PERSON
American NORP
Paul Hollywood PERSON
British NORP
Paul PERSON
